In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#   2. Load Dataset
df= pd.read_csv("data/UCI_Credit_Card.csv")

In [3]:
df.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [4]:
df.shape

(30000, 25)

In [5]:
#  3. Basic Sanity Checks
TARGET = "default.payment.next.month"
assert TARGET in df.columns, "Target column not found!"

In [6]:
df=df.drop_duplicates()

In [7]:
# 4. Missing Values Handling (Defensive)

# Separate feature types
cat_cols = ["SEX", "EDUCATION", "MARRIAGE"]
num_cols = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
num_cols = [x for x in num_cols if x not in cat_cols]
num_cols = [c for c in num_cols if c not in [TARGET]]

In [8]:
num_cols

['ID',
 'LIMIT_BAL',
 'AGE',
 'PAY_0',
 'PAY_2',
 'PAY_3',
 'PAY_4',
 'PAY_5',
 'PAY_6',
 'BILL_AMT1',
 'BILL_AMT2',
 'BILL_AMT3',
 'BILL_AMT4',
 'BILL_AMT5',
 'BILL_AMT6',
 'PAY_AMT1',
 'PAY_AMT2',
 'PAY_AMT3',
 'PAY_AMT4',
 'PAY_AMT5',
 'PAY_AMT6']

In [9]:
# Fill missing values with median (robust to outliers)
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


In [10]:
# ---- Categorical Features ----
for col in cat_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

In [11]:
# EDUCATION: group rare/invalid value
df["EDUCATION"] = df["EDUCATION"].replace([0, 5, 6], 4)

In [12]:
# MARRIAGE: invalid value
df["MARRIAGE"] = df["MARRIAGE"].replace(0, 3)

In [13]:
for col in cat_cols:
    df[col] = df[col].astype("int")

In [14]:
unique_MARRIAGE = df['MARRIAGE'].unique()
print(sorted(unique_MARRIAGE))

[np.int64(1), np.int64(2), np.int64(3)]


In [15]:
unique_EDUCATION = df['EDUCATION'].unique()
print(sorted(unique_EDUCATION))

[np.int64(1), np.int64(2), np.int64(3), np.int64(4)]


In [16]:
# 6. Outlier Handling (Winsorization)
# ======================================================
def outliers(series, lower=0.01, upper=0.99):
    low, high = series.quantile([lower, upper])
    return series.clip(low, high)

In [17]:
outlier_cols = [
    c for c in num_cols
    if c.startswith(("BILL_", "PAY_", "LIMIT_BAL"))
]
for col in outlier_cols:
    df[col] = outliers(df[col])

In [18]:
# 7. Feature Engineering
# ======================================================
bill_cols = [f"BILL_AMT{i}" for i in range(1, 7)]
pay_cols  = [f"PAY_AMT{i}" for i in range(1, 7)]
delay_cols = [f"PAY_{i}" for i in [0,2,3,4,5,6]]


In [19]:
# ---- Aggregations ----
df["TOTAL_BILL"] = df[bill_cols].sum(axis=1)
df["TOTAL_PAY"] = df[pay_cols].sum(axis=1)
df["PAY_RATIO"] = df["TOTAL_PAY"] / (df["TOTAL_BILL"] + 1)

In [20]:
# ---- Statistical Features ----
df["BILL_MEAN"] = df[bill_cols].mean(axis=1)
df["BILL_STD"]  = df[bill_cols].std(axis=1)
df["BILL_MAX"]  = df[bill_cols].max(axis=1)

df["PAY_MEAN"] = df[pay_cols].mean(axis=1)
df["PAY_STD"]  = df[pay_cols].std(axis=1)
df["PAY_MAX"]  = df[pay_cols].max(axis=1)

In [21]:
# ---- Behavior Features ----
df["NB_LATE_PAYMENTS"] = (df[delay_cols] > 0).sum(axis=1)

In [22]:
# ---- Trend Features ----
df["BILL_TREND"] = df["BILL_AMT6"] - df["BILL_AMT1"]
df["PAY_TREND"]  = df["PAY_AMT6"] - df["PAY_AMT1"]

In [23]:
# 8. Skewness Handling (Log Transform)
# ======================================================
skewed_cols = [
    c for c in df.columns
    if c.startswith(("BILL_", "PAY_", "TOTAL_"))
]

for col in skewed_cols:
    df[col] = df[col].clip(lower=0)
    df[col] = np.log1p(df[col])


In [24]:
df

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,PAY_RATIO,BILL_MEAN,BILL_STD,BILL_MAX,PAY_MEAN,PAY_STD,PAY_MAX,NB_LATE_PAYMENTS,BILL_TREND,PAY_TREND
0,1,20000.0,2,2,1,24,1.098612,1.098612,0.000000,0.0,...,0.085648,7.158514,7.474564,8.272315,4.752152,5.642910,6.536692,2,0.000000,0.000000
1,2,120000.0,2,2,2,26,0.000000,1.098612,0.000000,0.0,...,0.256791,7.954080,6.459854,8.147867,6.726633,6.625091,7.601402,2,6.363028,7.601402
2,3,90000.0,2,2,2,34,0.000000,0.000000,0.000000,0.0,...,0.102906,9.737620,8.710375,10.283293,7.516071,7.359350,8.517393,0,0.000000,8.155649
3,4,50000.0,2,2,1,37,0.000000,0.000000,0.000000,0.0,...,0.035617,10.559884,9.265472,10.805517,7.243513,6.171822,7.610853,0,0.000000,0.000000
4,5,50000.0,1,2,1,57,0.000000,0.000000,0.000000,0.0,...,0.431814,9.810504,9.275153,10.486708,9.194465,9.531498,10.510041,0,9.260558,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,29996,220000.0,1,3,1,39,0.000000,0.000000,0.000000,0.0,...,0.057005,11.702657,11.370191,12.247051,8.866817,8.823989,9.903538,0,0.000000,0.000000
29996,29997,150000.0,1,3,2,43,0.000000,0.000000,0.000000,0.0,...,0.521195,8.169431,8.071385,9.102755,7.789869,8.165228,9.104869,0,0.000000,0.000000
29997,29998,30000.0,1,2,2,37,1.386294,1.386294,1.098612,0.0,...,0.367410,9.371637,9.143682,9.946499,8.559806,9.034926,9.998843,3,9.667322,8.039480
29998,29999,80000.0,1,3,1,41,0.693147,0.000000,0.000000,0.0,...,0.389718,10.707658,10.390838,11.269324,9.966534,10.312097,11.105306,1,10.800106,0.000000


In [25]:
df.columns

Index(['ID', 'LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0',
       'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6',
       'default.payment.next.month', 'TOTAL_BILL', 'TOTAL_PAY', 'PAY_RATIO',
       'BILL_MEAN', 'BILL_STD', 'BILL_MAX', 'PAY_MEAN', 'PAY_STD', 'PAY_MAX',
       'NB_LATE_PAYMENTS', 'BILL_TREND', 'PAY_TREND'],
      dtype='object')

In [26]:
df.to_csv('data/data_clean.csv',index=False)